# 📖 Evolución de los Modelos de Lenguaje - Parte 2
## Secciones Avanzadas: RAG, Agentes y Conclusiones

**Nota**: Este notebook es continuación de `Evolucion_Modelos_Lenguaje.ipynb`

---

## Contenido:
- **Sección 8**: Mini-RAG (Retrieval Augmented Generation)
- **Sección 9**: Mini-Agente de IA Simulado
- **Sección 10**: Conclusiones y Recursos

---

## 8. Mini-RAG Introductorio

### ¿Qué es RAG?

**RAG** = **Retrieval Augmented Generation**  
(Generación Aumentada con Recuperación)

Es una técnica que combina:
1. **Búsqueda** de información relevante en una base de conocimientos
2. **Aumentación** del prompt con esa información como contexto
3. **Generación** de una respuesta basada en ese contexto

---

### ¿Por qué es importante RAG?

#### **Problema de los LLM tradicionales:**
- ❌ Conocimiento limitado a datos de entrenamiento (cortado en una fecha)
- ❌ No tienen acceso a información actualizada
- ❌ No pueden consultar bases de datos privadas
- ❌ **Alucinaciones**: inventan información que suena plausible pero es falsa

#### **Solución: RAG**
- ✅ Acceso a información actualizada
- ✅ Funciona con datos privados/específicos
- ✅ Reduce alucinaciones (respuestas basadas en fuentes reales)
- ✅ Trazabilidad: puedes citar las fuentes utilizadas

---

### Pipeline RAG Básico

```
1. Usuario hace una pregunta
   ↓
2. Generamos embedding de la pregunta
   ↓
3. Buscamos documentos similares en la base de conocimientos (usando similitud de embeddings)
   ↓
4. Seleccionamos los K documentos más relevantes
   ↓
5. Construimos un prompt que incluye:
   - Contexto: documentos recuperados
   - Pregunta del usuario
   - Instrucción: "Responde basándote en el contexto"
   ↓
6. Modelo genera respuesta usando el contexto
   ↓
7. Devolvemos la respuesta (+ opcionalmente las fuentes)
```

---

### Aplicaciones Reales

- **Chatbots empresariales**: Acceso a documentación interna
- **Asistentes médicos**: Consulta de literatura científica actualizada
- **Sistemas de soporte**: Base de conocimiento de productos
- **Asistentes legales**: Búsqueda en jurisprudencia
- **Educación**: Sistemas de tutoría con materiales específicos


### Implementación Práctica

In [1]:
# Importamos las librerías necesarias (si no las tienes del notebook anterior)
!pip install -q sentence-transformers transformers

In [2]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline
import numpy as np

# Cargamos los modelos
modelo_embeddings = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
generador = pipeline('text-generation', model='distilgpt2', device=-1)

print("✅ Modelos cargados")

/Users/imbricadas/mambaforge/envs/embeddings/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use cpu


✅ Modelos cargados


In [3]:
# Creamos una base de conocimiento sobre diferentes temas
base_conocimiento = [
    """Python es un lenguaje de programación de alto nivel creado por Guido van Rossum en 1991. 
    Se caracteriza por su sintaxis simple y legible, lo que lo hace ideal para principiantes. 
    Es ampliamente usado en ciencia de datos, desarrollo web, automatización y machine learning.""",
    
    """Machine Learning es una rama de la inteligencia artificial que permite a las computadoras 
    aprender de datos sin ser explícitamente programadas. Los algoritmos de ML identifican patrones 
    en los datos y hacen predicciones. Se divide en aprendizaje supervisado, no supervisado y por refuerzo.""",
    
    """BERT (Bidirectional Encoder Representations from Transformers) es un modelo de lenguaje 
    pre-entrenado desarrollado por Google en 2018. A diferencia de modelos anteriores, BERT lee 
    el texto de manera bidireccional, lo que le permite entender mejor el contexto. Es excelente 
    para tareas de comprensión como clasificación y question answering.""",
    
    """GPT (Generative Pre-trained Transformer) es una familia de modelos desarrollados por OpenAI. 
    GPT-3 tiene 175 mil millones de parámetros y puede generar texto coherente, traducir, responder 
    preguntas y hasta programar. GPT-4 es la versión más reciente y multimodal.""",
    
    """Los transformers revolucionaron el NLP en 2017 con el paper 'Attention Is All You Need'. 
    Utilizan mecanismos de atención para procesar secuencias completas en paralelo, superando 
    las limitaciones de las RNN. Son la base de modelos modernos como BERT, GPT y T5.""",
    
    """RAG (Retrieval Augmented Generation) combina búsqueda de información con generación de texto. 
    Primero busca documentos relevantes en una base de datos, luego usa esos documentos como contexto 
    para que el modelo genere una respuesta informada. Esto reduce las alucinaciones y permite 
    acceso a información actualizada."""
]

print("📚 Base de Conocimiento creada")
print(f"📊 Total de documentos: {len(base_conocimiento)}\n")

for i, doc in enumerate(base_conocimiento, 1):
    print(f"{i}. {doc[:80]}...")
    print()

📚 Base de Conocimiento creada
📊 Total de documentos: 6

1. Python es un lenguaje de programación de alto nivel creado por Guido van Rossum ...

2. Machine Learning es una rama de la inteligencia artificial que permite a las com...

3. BERT (Bidirectional Encoder Representations from Transformers) es un modelo de l...

4. GPT (Generative Pre-trained Transformer) es una familia de modelos desarrollados...

5. Los transformers revolucionaron el NLP en 2017 con el paper 'Attention Is All Yo...

6. RAG (Retrieval Augmented Generation) combina búsqueda de información con generac...



In [4]:
# Generamos embeddings de todos los documentos
embeddings_docs = modelo_embeddings.encode(base_conocimiento)

print(f"✅ Embeddings generados para {len(base_conocimiento)} documentos")
print(f"📊 Dimensión de cada embedding: {embeddings_docs.shape[1]}")

✅ Embeddings generados para 6 documentos
📊 Dimensión de cada embedding: 384


In [5]:
def buscar_documentos_relevantes(pregunta, top_k=2):
    """
    Busca los documentos más relevantes para una pregunta.
    
    Args:
        pregunta: Pregunta del usuario
        top_k: Número de documentos a recuperar
    
    Returns:
        Lista de tuplas (documento, similitud, índice)
    """
    # Generamos embedding de la pregunta
    embedding_pregunta = modelo_embeddings.encode([pregunta])
    
    # Calculamos similitud con todos los documentos
    similitudes = cosine_similarity(embedding_pregunta, embeddings_docs)[0]
    
    # Obtenemos los top_k más similares
    indices_top = np.argsort(similitudes)[::-1][:top_k]
    
    resultados = [
        (base_conocimiento[idx], similitudes[idx], idx) 
        for idx in indices_top
    ]
    
    return resultados

def sistema_rag(pregunta, usar_contexto=True, top_k=2):
    """
    Sistema RAG completo: búsqueda + generación.
    
    Args:
        pregunta: Pregunta del usuario
        usar_contexto: Si False, genera sin contexto (para comparar)
        top_k: Número de documentos a recuperar
    
    Returns:
        Respuesta generada, documentos usados
    """
    if usar_contexto:
        # Paso 1: Buscar documentos relevantes
        docs_relevantes = buscar_documentos_relevantes(pregunta, top_k=top_k)
        
        print(f"\n🔍 Documentos recuperados (similitud):")
        for i, (doc, sim, idx) in enumerate(docs_relevantes, 1):
            print(f"\n{i}. [Similitud: {sim:.3f}] Doc {idx+1}")
            print(f"   {doc[:100]}...")
        
        # Paso 2: Construir prompt con contexto
        contexto = "\n\n".join([doc for doc, _, _ in docs_relevantes])
        
        prompt = f"""Context information:
{contexto}

Question: {pregunta}

Answer based on the context above:"""
        
    else:
        # Sin contexto RAG
        prompt = f"Question: {pregunta}\n\nAnswer:"
        docs_relevantes = []
    
    # Paso 3: Generar respuesta
    respuesta = generador(
        prompt,
        max_length=150,
        temperature=0.7,
        num_return_sequences=1,
        do_sample=True,
        pad_token_id=50256
    )[0]['generated_text']
    
    return respuesta, docs_relevantes

print("✅ Sistema RAG definido")

✅ Sistema RAG definido


In [6]:
# Probamos el sistema RAG
pregunta_test = "What is BERT and what makes it special?"

print("="*90)
print(f"❓ Pregunta: {pregunta_test}")
print("="*90)

respuesta, docs = sistema_rag(pregunta_test, usar_contexto=True, top_k=2)

print("\n" + "="*90)
print("\n💬 Respuesta Generada (con RAG):\n")
print(respuesta)
print("\n" + "="*90)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


❓ Pregunta: What is BERT and what makes it special?

🔍 Documentos recuperados (similitud):

1. [Similitud: 0.519] Doc 3
   BERT (Bidirectional Encoder Representations from Transformers) es un modelo de lenguaje 
    pre-ent...

2. [Similitud: 0.225] Doc 5
   Los transformers revolucionaron el NLP en 2017 con el paper 'Attention Is All You Need'. 
    Utiliz...


💬 Respuesta Generada (con RAG):

Context information:
BERT (Bidirectional Encoder Representations from Transformers) es un modelo de lenguaje 
    pre-entrenado desarrollado por Google en 2018. A diferencia de modelos anteriores, BERT lee 
    el texto de manera bidireccional, lo que le permite entender mejor el contexto. Es excelente 
    para tareas de comprensión como clasificación y question answering.

Los transformers revolucionaron el NLP en 2017 con el paper 'Attention Is All You Need'. 
    Utilizan mecanismos de atención para procesar secuencias completas en paralelo, superando 
    las limitaciones de las RNN. Son la

### Comparación: Con RAG vs Sin RAG

In [7]:
# Comparamos respuestas con y sin RAG
pregunta = "Who created Python and when?"

print("="*90)
print(f"❓ Pregunta: {pregunta}")
print("="*90)

# CON RAG
print("\n🟢 CON RAG (usando contexto de la base de conocimiento):")
print("-"*90)
respuesta_rag, docs_rag = sistema_rag(pregunta, usar_contexto=True)
print("\nRespuesta:")
print(respuesta_rag[:300] + "...")

print("\n" + "="*90)

# SIN RAG
print("\n🔴 SIN RAG (sin contexto - solo conocimiento del modelo):")
print("-"*90)
respuesta_sin_rag, _ = sistema_rag(pregunta, usar_contexto=False)
print("\nRespuesta:")
print(respuesta_sin_rag[:300] + "...")

print("\n" + "="*90)
print("\n💡 Observa cómo RAG proporciona información más precisa basada en fuentes reales")

Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


❓ Pregunta: Who created Python and when?

🟢 CON RAG (usando contexto de la base de conocimiento):
------------------------------------------------------------------------------------------

🔍 Documentos recuperados (similitud):

1. [Similitud: 0.648] Doc 1
   Python es un lenguaje de programación de alto nivel creado por Guido van Rossum en 1991. 
    Se car...

2. [Similitud: 0.244] Doc 3
   BERT (Bidirectional Encoder Representations from Transformers) es un modelo de lenguaje 
    pre-ent...


Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



Respuesta:
Context information:
Python es un lenguaje de programación de alto nivel creado por Guido van Rossum en 1991. 
    Se caracteriza por su sintaxis simple y legible, lo que lo hace ideal para principiantes. 
    Es ampliamente usado en ciencia de datos, desarrollo web, automatización y machine learnin...


🔴 SIN RAG (sin contexto - solo conocimiento del modelo):
------------------------------------------------------------------------------------------

Respuesta:
Question: Who created Python and when?

Answer:
I'm not sure. I don't know. I'm just a programmer. I'm a programmer. I'm a programmer. My job is to get me to do what I like. I'm a programmer. I'm a programmer. I'm a programmer. I'm a programmer. I'm a programmer. My job is to get me to do what I lik...


💡 Observa cómo RAG proporciona información más precisa basada en fuentes reales


---

### 📝 EJERCICIO 20: Cambia la base de conocimiento

In [8]:
# 📝 EJERCICIO: Crea tu propia base de conocimiento sobre un tema de tu interés

# Define tu base de conocimiento aquí:
mi_base_conocimiento = [
    # Añade al menos 4-5 párrafos sobre un tema específico
    # Ejemplo tema: Historia de la computación, Cambio climático, Nutrición, etc.
    
]

# Define preguntas relacionadas:
mis_preguntas = [
    # Añade 3-5 preguntas que puedan responderse con tu base de conocimiento
    
]

if mi_base_conocimiento and mis_preguntas:
    # Generamos embeddings
    mis_embeddings = modelo_embeddings.encode(mi_base_conocimiento)
    
    # Función RAG adaptada
    def mi_rag(pregunta):
        emb_pregunta = modelo_embeddings.encode([pregunta])
        similitudes = cosine_similarity(emb_pregunta, mis_embeddings)[0]
        idx_mejor = np.argmax(similitudes)
        
        contexto = mi_base_conocimiento[idx_mejor]
        prompt = f"Context: {contexto}\n\nQuestion: {pregunta}\n\nAnswer:"
        
        respuesta = generador(
            prompt, max_length=120, temperature=0.7, do_sample=True, pad_token_id=50256
        )[0]['generated_text']
        
        return respuesta, contexto, similitudes[idx_mejor]
    
    # Probamos con tus preguntas
    for i, pregunta in enumerate(mis_preguntas, 1):
        print(f"\n{'='*90}")
        print(f"Pregunta {i}: {pregunta}")
        print(f"{'='*90}")
        
        respuesta, ctx, sim = mi_rag(pregunta)
        
        print(f"\n🔍 Contexto usado (similitud: {sim:.3f}):")
        print(f"   {ctx[:150]}...")
        print(f"\n💬 Respuesta:\n{respuesta}")
else:
    print("⚠️ Completa 'mi_base_conocimiento' y 'mis_preguntas'")
    print("\n💡 Sugerencias de temas:")
    print("   - Tu área de estudio o investigación")
    print("   - Hobby o interés personal (música, deportes, cocina)")
    print("   - Temas de actualidad que conozcas bien")

⚠️ Completa 'mi_base_conocimiento' y 'mis_preguntas'

💡 Sugerencias de temas:
   - Tu área de estudio o investigación
   - Hobby o interés personal (música, deportes, cocina)
   - Temas de actualidad que conozcas bien


### 📝 EJERCICIO 21: Comparación con y sin contexto

In [9]:
# 📝 EJERCICIO: Analiza cómo cambia la calidad con/sin RAG

preguntas_comparacion = [
    "What are transformers in NLP?",
    "Explain the difference between BERT and GPT",
    "What is RAG and why is it useful?"
]

print("📊 Análisis Comparativo: RAG vs Sin RAG\n")
print("="*90)

for i, pregunta in enumerate(preguntas_comparacion, 1):
    print(f"\n\n{'#'*90}")
    print(f"PREGUNTA {i}: {pregunta}")
    print(f"{'#'*90}")
    
    # Con RAG
    print("\n✅ CON RAG:")
    resp_rag, docs = sistema_rag(pregunta, usar_contexto=True, top_k=1)
    print(f"\nRespuesta: {resp_rag[:250]}...")
    
    print("\n" + "-"*90)
    
    # Sin RAG
    print("\n❌ SIN RAG:")
    resp_sin, _ = sistema_rag(pregunta, usar_contexto=False)
    print(f"\nRespuesta: {resp_sin[:250]}...")

print("\n\n" + "="*90)
print("\n💡 Reflexión:")
print("   ¿Cuál versión proporciona información más precisa?")
print("   ¿El modelo sin RAG 'alucina' o inventa información?")
print("   ¿Cómo se podría mejorar el sistema RAG?")

Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


📊 Análisis Comparativo: RAG vs Sin RAG



##########################################################################################
PREGUNTA 1: What are transformers in NLP?
##########################################################################################

✅ CON RAG:

🔍 Documentos recuperados (similitud):

1. [Similitud: 0.661] Doc 5
   Los transformers revolucionaron el NLP en 2017 con el paper 'Attention Is All You Need'. 
    Utiliz...


Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



Respuesta: Context information:
Los transformers revolucionaron el NLP en 2017 con el paper 'Attention Is All You Need'. 
    Utilizan mecanismos de atención para procesar secuencias completas en paralelo, superando 
    las limitaciones de las RNN. Son la base...

------------------------------------------------------------------------------------------

❌ SIN RAG:


Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...


Respuesta: Context information:
GPT (Generative Pre-trained Transformer) es una familia de modelos desarrollados por OpenAI. 
    GPT-3 tiene 175 mil millones de parámetros y puede generar texto coherente, traducir, responder 
    preguntas y hasta programar. G...

------------------------------------------------------------------------------------------

❌ SIN RAG:


Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



Respuesta: Question: Explain the difference between BERT and GPT

Answer: This question is not to be confused with BERT
Answer: The difference between BERT and GPT
Answer: The difference between BERT and GPT
Answer: The difference between BERT and GPT
Answer: T...


##########################################################################################
PREGUNTA 3: What is RAG and why is it useful?
##########################################################################################

✅ CON RAG:

🔍 Documentos recuperados (similitud):

1. [Similitud: 0.610] Doc 6
   RAG (Retrieval Augmented Generation) combina búsqueda de información con generación de texto. 
    P...


Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



Respuesta: Context information:
RAG (Retrieval Augmented Generation) combina búsqueda de información con generación de texto. 
    Primero busca documentos relevantes en una base de datos, luego usa esos documentos como contexto 
    para que el modelo genere u...

------------------------------------------------------------------------------------------

❌ SIN RAG:

Respuesta: Question: What is RAG and why is it useful?

Answer: The RAG's purpose is to help people understand the importance of RAG and why it should be used.
Question: Which RAGs help people understand?
Answer: RAG is a set of six different domains that help ...



💡 Reflexión:
   ¿Cuál versión proporciona información más precisa?
   ¿El modelo sin RAG 'alucina' o inventa información?
   ¿Cómo se podría mejorar el sistema RAG?


### 📝 EJERCICIO 22: Análisis de errores del modelo

In [10]:
# 📝 EJERCICIO: Identifica cuándo el modelo comete errores

# Preguntas que NO están en la base de conocimiento
preguntas_fuera_dominio = [
    "What is the capital of France?",
    "How do you cook pasta?",
    "Who won the World Cup in 2022?"
]

print("🔍 Análisis de Errores: Preguntas fuera del dominio\n")
print("="*90)
print("\n⚠️ Estas preguntas NO están cubiertas en nuestra base de conocimiento")
print("   Observa qué hace el sistema RAG en estos casos\n")

for i, pregunta in enumerate(preguntas_fuera_dominio, 1):
    print(f"\n{'='*90}")
    print(f"Pregunta {i}: {pregunta}")
    print(f"{'='*90}")
    
    respuesta, docs = sistema_rag(pregunta, usar_contexto=True, top_k=1)
    
    print(f"\n💬 Respuesta generada:\n{respuesta[:200]}...")
    
    print(f"\n❓ ¿El contexto recuperado es relevante para la pregunta?")
    if docs:
        sim = docs[0][1]
        print(f"   Similitud del mejor documento: {sim:.3f}")
        if sim < 0.3:
            print("   ⚠️ ALERTA: Similitud muy baja - probablemente respuesta no confiable")

print("\n\n" + "="*90)
print("\n💡 Mejoras posibles:")
print("   1. Definir umbral de similitud mínimo")
print("   2. Si similitud < umbral → Responder 'No tengo información suficiente'")
print("   3. Ampliar la base de conocimiento")
print("   4. Implementar re-ranking de documentos")

Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


🔍 Análisis de Errores: Preguntas fuera del dominio


⚠️ Estas preguntas NO están cubiertas en nuestra base de conocimiento
   Observa qué hace el sistema RAG en estos casos


Pregunta 1: What is the capital of France?

🔍 Documentos recuperados (similitud):

1. [Similitud: 0.005] Doc 1
   Python es un lenguaje de programación de alto nivel creado por Guido van Rossum en 1991. 
    Se car...


Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



💬 Respuesta generada:
Context information:
Python es un lenguaje de programación de alto nivel creado por Guido van Rossum en 1991. 
    Se caracteriza por su sintaxis simple y legible, lo que lo hace ideal para principian...

❓ ¿El contexto recuperado es relevante para la pregunta?
   Similitud del mejor documento: 0.005
   ⚠️ ALERTA: Similitud muy baja - probablemente respuesta no confiable

Pregunta 2: How do you cook pasta?

🔍 Documentos recuperados (similitud):

1. [Similitud: 0.109] Doc 3
   BERT (Bidirectional Encoder Representations from Transformers) es un modelo de lenguaje 
    pre-ent...


Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



💬 Respuesta generada:
Context information:
BERT (Bidirectional Encoder Representations from Transformers) es un modelo de lenguaje 
    pre-entrenado desarrollado por Google en 2018. A diferencia de modelos anteriores, BER...

❓ ¿El contexto recuperado es relevante para la pregunta?
   Similitud del mejor documento: 0.109
   ⚠️ ALERTA: Similitud muy baja - probablemente respuesta no confiable

Pregunta 3: Who won the World Cup in 2022?

🔍 Documentos recuperados (similitud):

1. [Similitud: 0.108] Doc 4
   GPT (Generative Pre-trained Transformer) es una familia de modelos desarrollados por OpenAI. 
    GP...

💬 Respuesta generada:
Context information:
GPT (Generative Pre-trained Transformer) es una familia de modelos desarrollados por OpenAI. 
    GPT-3 tiene 175 mil millones de parámetros y puede generar texto coherente, tradu...

❓ ¿El contexto recuperado es relevante para la pregunta?
   Similitud del mejor documento: 0.108
   ⚠️ ALERTA: Similitud muy baja - probablemente respuesta 

---

## 9. Mini-Agente de IA Simulado

### ¿Qué es un Agente de IA?

Un **agente de IA** es un sistema autónomo capaz de:

1. **Percibir**: Recibir un objetivo o tarea
2. **Planificar**: Descomponer la tarea en pasos
3. **Actuar**: Ejecutar acciones usando herramientas
4. **Aprender**: Ajustar según resultados

---

### Componentes de un Agente

#### **1. Objetivo (Goal)**
```
"Organizar un viaje a París para 3 personas en julio"
```

#### **2. Planificación**
```
1. Buscar vuelos disponibles
2. Comparar precios de hoteles
3. Crear itinerario de actividades
4. Calcular presupuesto total
```

#### **3. Herramientas/Acciones**
```python
- buscar_vuelos(origen, destino, fecha)
- buscar_hoteles(ciudad, fechas, personas)
- buscar_actividades(ciudad)
- calcular_costos(lista_items)
```

#### **4. Ejecución**
El agente ejecuta cada paso del plan usando las herramientas disponibles

---

### Tipos de Agentes

| Tipo | Descripción | Ejemplo |
|------|-------------|----------|
| **Reactivo** | Responde a estímulos inmediatos | Chatbot simple |
| **Planificador** | Crea plan antes de actuar | Asistente de viajes |
| **Aprendizaje** | Mejora con experiencia | Agente de RL |
| **Multi-agente** | Colabora con otros agentes | Sistema distribuido |

---

### Aplicaciones Reales

- **AutoGPT**: Agente que se auto-asigna sub-tareas
- **BabyAGI**: Agente con gestión de tareas
- **LangChain Agents**: Framework para construir agentes
- **ChatGPT con plugins**: Agente con acceso a herramientas

---

### Implementación Simple

Vamos a crear un agente MUY básico que:
1. Recibe un objetivo
2. Usa el modelo generativo para crear un plan
3. Simula la ejecución de cada paso

---

In [11]:
# Definimos funciones "dummy" que simulan herramientas del agente

def buscar_informacion(tema):
    """Simula búsqueda en internet"""
    print(f"  🔍 [HERRAMIENTA] Buscando información sobre: '{tema}'")
    print(f"     ✓ Búsqueda completada (simulado)")
    return f"Información encontrada sobre {tema}"

def leer_archivo(nombre_archivo):
    """Simula lectura de archivo"""
    print(f"  📄 [HERRAMIENTA] Leyendo archivo: '{nombre_archivo}'")
    print(f"     ✓ Archivo leído (simulado)")
    return f"Contenido de {nombre_archivo}"

def guardar_resultado(datos, archivo):
    """Simula guardar datos en archivo"""
    print(f"  💾 [HERRAMIENTA] Guardando datos en: '{archivo}'")
    print(f"     ✓ Datos guardados (simulado)")
    return f"Datos guardados en {archivo}"

def enviar_notificacion(mensaje):
    """Simula envío de notificación"""
    print(f"  📧 [HERRAMIENTA] Enviando notificación: '{mensaje[:50]}...'")
    print(f"     ✓ Notificación enviada (simulado)")
    return "Notificación enviada"

def analizar_datos(datos):
    """Simula análisis de datos"""
    print(f"  📊 [HERRAMIENTA] Analizando datos...")
    print(f"     ✓ Análisis completado (simulado)")
    return "Resultados del análisis"

print("✅ Herramientas del agente definidas")

✅ Herramientas del agente definidas


In [12]:
def planificar_objetivo(objetivo):
    """
    Usa el modelo generativo para crear un plan de acción.
    
    Args:
        objetivo: Descripción del objetivo a lograr
    
    Returns:
        Lista de pasos del plan
    """
    prompt = f"""Create a step-by-step action plan to achieve this goal: {objetivo}

Format the plan as numbered steps:
1. [First action]
2. [Second action]
3. [Third action]
4. [Fourth action]

Keep it practical and focused (maximum 5 steps).

Action Plan:"""
    
    plan_texto = generador(
        prompt,
        max_length=150,
        temperature=0.7,
        do_sample=True,
        pad_token_id=50256
    )[0]['generated_text']
    
    # Extraemos los pasos del plan
    pasos = []
    for linea in plan_texto.split('\n'):
        linea = linea.strip()
        # Buscamos líneas que empiecen con número
        if linea and (linea[0].isdigit() or linea.startswith('-')):
            # Limpiamos el paso
            paso = linea.lstrip('0123456789.-) ').strip()
            if paso and len(paso) > 10:  # Filtramos pasos muy cortos
                pasos.append(paso)
    
    # Si no encontramos pasos estructurados, devolvemos el texto completo dividido
    if not pasos:
        pasos = [plan_texto]
    
    return pasos[:5]  # Máximo 5 pasos

def ejecutar_paso(paso, num_paso):
    """
    Ejecuta un paso del plan llamando a la herramienta apropiada.
    
    Args:
        paso: Descripción del paso
        num_paso: Número del paso
    """
    print(f"\n📌 Ejecutando Paso {num_paso}: {paso}")
    print(f"{'─'*85}")
    
    paso_lower = paso.lower()
    
    # Lógica simple para decidir qué herramienta usar
    if any(palabra in paso_lower for palabra in ['search', 'find', 'look', 'research']):
        buscar_informacion(paso)
    elif any(palabra in paso_lower for palabra in ['read', 'review', 'check', 'analyze']):
        if 'data' in paso_lower or 'analyze' in paso_lower:
            analizar_datos(paso)
        else:
            leer_archivo("documento.txt")
    elif any(palabra in paso_lower for palabra in ['save', 'write', 'create', 'generate']):
        guardar_resultado(paso, "resultado.txt")
    elif any(palabra in paso_lower for palabra in ['send', 'notify', 'email', 'message']):
        enviar_notificacion(paso)
    else:
        print(f"  ⚙️ [ACCIÓN GENÉRICA] Ejecutando: {paso[:60]}...")
        print(f"     ✓ Completado")

def agente_autonomo(objetivo):
    """
    Agente completo: planifica y ejecuta de forma autónoma.
    
    Args:
        objetivo: Objetivo a lograr
    """
    print("\n" + "="*90)
    print("🤖 AGENTE DE IA AUTÓNOMO")
    print("="*90)
    print(f"\n🎯 OBJETIVO: {objetivo}")
    print("\n" + "─"*90)
    
    # Fase 1: Planificación
    print("\n🧠 FASE 1: PLANIFICACIÓN")
    print("─"*90)
    print("\nGenerando plan de acción...\n")
    
    pasos = planificar_objetivo(objetivo)
    
    print("📋 PLAN GENERADO:")
    for i, paso in enumerate(pasos, 1):
        print(f"   {i}. {paso}")
    
    # Fase 2: Ejecución
    print("\n" + "─"*90)
    print("\n⚙️ FASE 2: EJECUCIÓN")
    print("─"*90)
    
    for i, paso in enumerate(pasos, 1):
        ejecutar_paso(paso, i)
    
    # Finalización
    print("\n" + "="*90)
    print("✅ AGENTE COMPLETADO - Objetivo alcanzado")
    print("="*90 + "\n")

print("✅ Sistema de agente definido")

✅ Sistema de agente definido


In [13]:
# Probamos el agente con un objetivo
objetivo_ejemplo = "Research and summarize the latest trends in artificial intelligence"

agente_autonomo(objetivo_ejemplo)

Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



🤖 AGENTE DE IA AUTÓNOMO

🎯 OBJETIVO: Research and summarize the latest trends in artificial intelligence

──────────────────────────────────────────────────────────────────────────────────────────

🧠 FASE 1: PLANIFICACIÓN
──────────────────────────────────────────────────────────────────────────────────────────

Generando plan de acción...

📋 PLAN GENERADO:
   1. [First action]
   2. [Second action]
   3. [Third action]
   4. [Fourth action]
   5. [Second action]

──────────────────────────────────────────────────────────────────────────────────────────

⚙️ FASE 2: EJECUCIÓN
──────────────────────────────────────────────────────────────────────────────────────────

📌 Ejecutando Paso 1: [First action]
─────────────────────────────────────────────────────────────────────────────────────
  ⚙️ [ACCIÓN GENÉRICA] Ejecutando: [First action]...
     ✓ Completado

📌 Ejecutando Paso 2: [Second action]
─────────────────────────────────────────────────────────────────────────────────────
  ⚙️ [AC

---

### 📝 EJERCICIO 23: Cambia el objetivo del agente

In [14]:
# 📝 EJERCICIO: Define tu propio objetivo y observa cómo el agente lo descompone

# Define tu objetivo aquí:
mi_objetivo = ""  # Ejemplos:
                   # "Create a marketing campaign for a new product"
                   # "Plan a healthy meal prep for the week"
                   # "Learn Python programming in 30 days"

if mi_objetivo:
    agente_autonomo(mi_objetivo)
else:
    print("⚠️ Define un objetivo en la variable 'mi_objetivo'")
    print("\n💡 Ejemplos de objetivos:")
    print("   - 'Organize a team-building event for 20 people'")
    print("   - 'Write and publish a blog post about climate change'")
    print("   - 'Prepare a budget analysis for next quarter'")
    print("   - 'Learn to play guitar basics in one month'")

⚠️ Define un objetivo en la variable 'mi_objetivo'

💡 Ejemplos de objetivos:
   - 'Organize a team-building event for 20 people'
   - 'Write and publish a blog post about climate change'
   - 'Prepare a budget analysis for next quarter'
   - 'Learn to play guitar basics in one month'


### 📝 EJERCICIO 24: Añade una nueva herramienta

In [15]:
# 📝 EJERCICIO: Crea tu propia herramienta y añádela al agente

def mi_herramienta_personalizada(parametro):
    """
    Define aquí tu herramienta personalizada.
    
    Ejemplos:
    - calcular_costos()
    - traducir_texto()
    - generar_reporte()
    - enviar_recordatorio()
    """
    print(f"  🔧 [MI HERRAMIENTA] Ejecutando acción personalizada...")
    print(f"     Parámetro: {parametro[:50]}...")
    print(f"     ✓ Acción completada")
    return "Resultado de mi herramienta"

def ejecutar_paso_mejorado(paso, num_paso):
    """Versión mejorada con tu herramienta incluida"""
    print(f"\n📌 Ejecutando Paso {num_paso}: {paso}")
    print(f"{'─'*85}")
    
    paso_lower = paso.lower()
    
    # AÑADE AQUÍ LA CONDICIÓN PARA TU HERRAMIENTA
    # Ejemplo:
    # if 'palabra_clave' in paso_lower:
    #     mi_herramienta_personalizada(paso)
    
    # Herramientas existentes
    if any(palabra in paso_lower for palabra in ['search', 'find', 'research']):
        buscar_informacion(paso)
    elif any(palabra in paso_lower for palabra in ['read', 'review', 'check']):
        leer_archivo("documento.txt")
    elif any(palabra in paso_lower for palabra in ['save', 'write', 'create']):
        guardar_resultado(paso, "resultado.txt")
    elif any(palabra in paso_lower for palabra in ['send', 'notify']):
        enviar_notificacion(paso)
    # AÑADE TU CONDICIÓN AQUÍ
    
    else:
        print(f"  ⚙️ [ACCIÓN GENÉRICA] {paso[:60]}...")
        print(f"     ✓ Completado")

def agente_mejorado(objetivo):
    """Agente que usa tu herramienta personalizada"""
    print("\n" + "="*90)
    print("🤖 AGENTE MEJORADO (con herramienta personalizada)")
    print("="*90)
    print(f"\n🎯 OBJETIVO: {objetivo}\n")
    
    pasos = planificar_objetivo(objetivo)
    
    print("📋 PLAN:")
    for i, paso in enumerate(pasos, 1):
        print(f"   {i}. {paso}")
    
    print("\n" + "─"*90)
    print("\n⚙️ EJECUTANDO:\n")
    
    for i, paso in enumerate(pasos, 1):
        ejecutar_paso_mejorado(paso, i)
    
    print("\n" + "="*90)
    print("✅ COMPLETADO")
    print("="*90 + "\n")

# Prueba tu agente mejorado:
# agente_mejorado("Tu objetivo aquí")

print("✅ Agente mejorado definido")
print("\n💡 Completa:")
print("   1. La función 'mi_herramienta_personalizada'")
print("   2. La condición en 'ejecutar_paso_mejorado'")
print("   3. Prueba con un objetivo que active tu herramienta")

✅ Agente mejorado definido

💡 Completa:
   1. La función 'mi_herramienta_personalizada'
   2. La condición en 'ejecutar_paso_mejorado'
   3. Prueba con un objetivo que active tu herramienta


### 📝 EJERCICIO 25: Identifica decisiones incorrectas

In [16]:
# 📝 EJERCICIO: Analiza cuándo el agente toma decisiones subóptimas

objetivos_desafiantes = [
    "Build a house from scratch",
    "Become a professional athlete in one week",
    "Solve world hunger"
]

print("🔍 Análisis de Decisiones del Agente\n")
print("="*90)
print("\nProbando con objetivos POCO REALISTAS o DEMASIADO COMPLEJOS")
print("Observa si el agente:")
print("  - Genera planes realistas")
print("  - Identifica limitaciones")
print("  - Divide apropiadamente tareas complejas\n")

for i, objetivo in enumerate(objetivos_desafiantes, 1):
    print(f"\n{'#'*90}")
    print(f"CASO {i}: {objetivo}")
    print(f"{'#'*90}")
    
    pasos = planificar_objetivo(objetivo)
    
    print("\n📋 Plan generado:")
    for j, paso in enumerate(pasos, 1):
        print(f"   {j}. {paso}")
    
    print("\n🤔 Análisis:")
    print("   ¿El plan es realista?")
    print("   ¿Falta algún paso importante?")
    print("   ¿Los pasos son demasiado vagos o específicos?")

print("\n\n" + "="*90)
print("\n💡 Limitaciones observadas:")
print("   - El modelo es pequeño (DistilGPT-2) → planes simples")
print("   - No tiene sentido común real → puede ser poco realista")
print("   - No verifica viabilidad → acepta cualquier objetivo")
print("   - No tiene memoria → cada plan es independiente")
print("\n🚀 Mejoras posibles:")
print("   - Usar modelos más grandes (GPT-4, Claude)")
print("   - Añadir validación de viabilidad")
print("   - Implementar memoria de acciones previas")
print("   - Permitir replanning si una acción falla")

Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


🔍 Análisis de Decisiones del Agente


Probando con objetivos POCO REALISTAS o DEMASIADO COMPLEJOS
Observa si el agente:
  - Genera planes realistas
  - Identifica limitaciones
  - Divide apropiadamente tareas complejas


##########################################################################################
CASO 1: Build a house from scratch
##########################################################################################


Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



📋 Plan generado:
   1. [First action]
   2. [Second action]
   3. [Third action]
   4. [Fourth action]
   5. [Second action]

🤔 Análisis:
   ¿El plan es realista?
   ¿Falta algún paso importante?
   ¿Los pasos son demasiado vagos o específicos?

##########################################################################################
CASO 2: Become a professional athlete in one week
##########################################################################################


Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



📋 Plan generado:
   1. [First action]
   2. [Second action]
   3. [Third action]
   4. [Fourth action]

🤔 Análisis:
   ¿El plan es realista?
   ¿Falta algún paso importante?
   ¿Los pasos son demasiado vagos o específicos?

##########################################################################################
CASO 3: Solve world hunger
##########################################################################################

📋 Plan generado:
   1. [First action]
   2. [Second action]
   3. [Third action]
   4. [Fourth action]
   5. [Second action]

🤔 Análisis:
   ¿El plan es realista?
   ¿Falta algún paso importante?
   ¿Los pasos son demasiado vagos o específicos?



💡 Limitaciones observadas:
   - El modelo es pequeño (DistilGPT-2) → planes simples
   - No tiene sentido común real → puede ser poco realista
   - No verifica viabilidad → acepta cualquier objetivo
   - No tiene memoria → cada plan es independiente

🚀 Mejoras posibles:
   - Usar modelos más grandes (GPT-4, Claude)


---

## 10. Conclusiones y Recursos

### 🎓 Resumen del Aprendizaje

¡Felicidades! Has completado un recorrido completo por la evolución de los modelos de lenguaje.

---

### ✅ Lo que has aprendido:

#### **1. Fundamentos Históricos**
- Evolución del NLP: reglas → estadística → deep learning → transformers → LLMs
- Cada era resolvió limitaciones de la anterior

#### **2. Técnicas Clásicas de NLP**
- ✅ Tokenización y normalización de texto
- ✅ Stemming vs Lematización
- ✅ N-gramas para capturar contexto local
- ✅ Análisis de frecuencias

#### **3. Representaciones Vectoriales**
- ✅ Bag of Words (BoW): frecuencias simples
- ✅ TF-IDF: ponderación por importancia
- ✅ Limitaciones: no capturan semántica

#### **4. Embeddings Modernos**
- ✅ Word2Vec, GloVe: primeros embeddings semánticos
- ✅ Sentence Transformers: embeddings de frases completas
- ✅ Similitud del coseno para comparar significados
- ✅ Aplicaciones: búsqueda semántica, clustering, clasificación

#### **5. Arquitecturas Transformer**
- ✅ **Encoder (BERT)**: Comprensión bidireccional → Clasificación, NER
- ✅ **Decoder (GPT)**: Generación autoregresiva → Chatbots, escritura
- ✅ **Encoder-Decoder (T5)**: Transformación → Traducción, resúmenes

#### **6. Modelos Generativos**
- ✅ Predicción del siguiente token
- ✅ Parámetros: temperature, top-k, top-p, max_length
- ✅ Balance creatividad vs coherencia

#### **7. Prompt Engineering**
- ✅ Principios: contexto, rol, formato, ejemplos, restricciones
- ✅ Zero-shot, one-shot, few-shot learning
- ✅ Chain-of-Thought reasoning

#### **8. RAG (Retrieval Augmented Generation)**
- ✅ Pipeline: búsqueda → aumentación → generación
- ✅ Reduce alucinaciones usando fuentes reales
- ✅ Permite actualizar conocimiento sin reentrenar

#### **9. Agentes de IA**
- ✅ Componentes: percepción, planificación, ejecución
- ✅ Herramientas y acciones
- ✅ Autonomía y toma de decisiones

---

### 📊 Habilidades Técnicas Adquiridas

**Librerías y frameworks:**
- ✅ `spaCy`: Procesamiento lingüístico
- ✅ `NLTK`: Análisis de texto clásico
- ✅ `scikit-learn`: Vectorización y ML
- ✅ `transformers`: Modelos pre-entrenados
- ✅ `sentence-transformers`: Embeddings semánticos
- ✅ `matplotlib`: Visualización

**Conceptos de ML/DL:**
- ✅ Embeddings y espacios vectoriales
- ✅ Similitud del coseno
- ✅ Reducción dimensional (PCA)
- ✅ Clustering (K-means)
- ✅ Clasificación de texto

---

### 🚀 Próximos Pasos

#### **1. Profundiza en Fundamentos**
```python
# Temas a explorar:
- Arquitectura de Transformers en detalle
- Mecanismo de atención (self-attention)
- Tokenización avanzada (BPE, WordPiece)
- Fine-tuning de modelos
```

#### **2. Proyectos Prácticos**
```
Nivel Principiante:
→ Clasificador de sentimientos para reviews de productos
→ Sistema de búsqueda semántica para documentos
→ Chatbot con reglas + modelo generativo

Nivel Intermedio:
→ Sistema RAG para documentación técnica
→ Resumidor automático de artículos
→ Extractor de entidades nombradas (NER)

Nivel Avanzado:
→ Fine-tuning de modelos para dominio específico
→ Multi-agente con LangChain
→ Sistema de QA con múltiples fuentes
```

#### **3. Frameworks y Herramientas**

**LangChain** (https://langchain.com)
```python
# Framework para aplicaciones con LLMs
- Chains: Secuencias de operaciones
- Agents: Sistemas autónomos con herramientas
- Memory: Gestión de contexto conversacional
- VectorStores: Bases de datos vectoriales
```

**LlamaIndex** (https://llamaindex.ai)
```python
# Especializado en RAG e indexación
- Indexación eficiente de documentos
- Query engines avanzados
- Integración con múltiples fuentes
```

**Haystack** (https://haystack.deepset.ai)
```python
# Framework end-to-end para NLP
- Pipelines de búsqueda
- Question Answering
- Generación aumentada
```

---

### 📚 Recursos Recomendados

#### **Documentación Oficial**

| Recurso | URL | Descripción |
|---------|-----|-------------|
| Hugging Face Docs | https://huggingface.co/docs | Modelos, datasets, tutoriales |
| Transformers Docs | https://huggingface.co/docs/transformers | API de transformers |
| Sentence Transformers | https://www.sbert.net | Embeddings de frases |
| spaCy | https://spacy.io | NLP industrial |
| LangChain | https://python.langchain.com | Aplicaciones con LLMs |

#### **Cursos Online (Gratuitos)**

**1. Hugging Face Course**  
https://huggingface.co/course  
- Completo y práctico
- Desde básico hasta avanzado
- Certificado gratuito

**2. Fast.ai NLP**  
https://www.fast.ai/  
- Enfoque top-down (práctica primero)
- Código en PyTorch

**3. DeepLearning.AI Short Courses**  
https://www.deeplearning.ai/short-courses/  
- Cursos de 1-2 horas
- Temas específicos (RAG, Agents, Fine-tuning)
- Gratuitos

**4. Stanford CS224N**  
http://web.stanford.edu/class/cs224n/  
- Curso universitario completo
- Videos, slides, tareas
- Nivel académico riguroso

#### **Papers Fundamentales**

**Deben leer:**

1. **"Attention Is All You Need"** (2017)  
   https://arxiv.org/abs/1706.03762  
   → Paper original de Transformers

2. **"BERT: Pre-training of Deep Bidirectional Transformers"** (2018)  
   https://arxiv.org/abs/1810.04805  
   → BERT y masked language modeling

3. **"Language Models are Few-Shot Learners"** (GPT-3, 2020)  
   https://arxiv.org/abs/2005.14165  
   → Capacidades de modelos grandes

4. **"Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks"** (2020)  
   https://arxiv.org/abs/2005.11401  
   → RAG original

#### **Libros**

**1. "Natural Language Processing with Transformers"**  
por Lewis Tunstall, Leandro von Werra, Thomas Wolf  
- Práct ico y actualizado
- Código con Hugging Face

**2. "Speech and Language Processing"**  
por Dan Jurafsky y James H. Martin  
- Gratuito online: https://web.stanford.edu/~jurafsky/slp3/
- Fundamentos sólidos

**3. "Dive into Deep Learning"**  
https://d2l.ai  
- Interactivo (Jupyter notebooks)
- Gratuito

#### **Comunidades y Foros**

- **Hugging Face Forum**: https://discuss.huggingface.co/
- **r/MachineLearning**: https://reddit.com/r/MachineLearning
- **r/LanguageTechnology**: https://reddit.com/r/LanguageTechnology
- **Discord de Hugging Face**: https://hf.co/join/discord
- **Papers With Code**: https://paperswithcode.com/area/natural-language-processing

#### **Blogs y Newsletters**

- **The Batch** (deeplearning.ai): https://www.deeplearning.ai/the-batch/
- **Hugging Face Blog**: https://huggingface.co/blog
- **Distill**: https://distill.pub/
- **Sebastian Ruder's Blog**: https://ruder.io/
- **Jay Alammar's Blog**: https://jalammar.github.io/

---